<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://vespa.ai/assets/vespa-ai-logo-heather.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://vespa.ai/assets/vespa-ai-logo-rock.svg">
  <img alt="#Vespa" width="200" src="https://vespa.ai/assets/vespa-ai-logo-rock.svg" style="margin-bottom: 25px;">
</picture>

# Authenticating to Vespa Cloud

Security is a top priority for the Vespa Team.
We understand that as a newcomer to Vespa, the different authentication methods may not always be immediately clear.

This notebook is intended to provide some clarity on the different authentication methods needing when interacting with Vespa Cloud for different purposes.


<div class="alert alert-info">
    Refer to <a href="https://pyvespa.readthedocs.io/en/latest/troubleshooting.html">troubleshooting</a>
    for any problem when running this guide.
</div>


**Pre-requisite**: Create a tenant at [cloud.vespa.ai](https://cloud.vespa.ai/), save the tenant name.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vespa-engine/pyvespa/blob/master/docs/sphinx/source/getting-started-pyvespa-cloud.ipynb)


## Install

Install [pyvespa](https://pyvespa.readthedocs.io/) >= 0.45
and the [Vespa CLI](https://docs.vespa.ai/en/vespa-cli.html).


In [ ]:
!pip3 install pyvespa vespacli

For background context, it is useful to read the [Vespa Cloud Security Guide](https://cloud.vespa.ai/en/security/guide).

## Control-plane vs Data-plane

|                                                                                              | Control-plane | Data-plane | Comments                                                                                   |
| -------------------------------------------------------------------------------------------- | ------------- | ---------- | ------------------------------------------------------------------------------------------ |
| Deploy application                                                                           | ✅            | ❌         |                                                                                            |
| Modify application (re-deploy)                                                               | ✅            | ❌         |                                                                                            |
| Add or modify data-plane certs or token(s)                                                   | ✅            | ❌         |                                                                                            |
| Feed data                                                                                    | ❌            | ✅         |                                                                                            |
| Query data                                                                                   | ❌            | ✅         |                                                                                            |
| Delete data                                                                                  | ❌            | ✅         |                                                                                            |
| [Visiting](https://docs.vespa.ai/en/visiting.html)                                           | ❌            | ✅         |                                                                                            |
| Get application package                                                                      | ✅            | ❌         |                                                                                            |
| [`vespa auth login`](https://docs.vespa.ai/en/reference/vespa-cli/vespa_auth_login.html)     | ✅            | ❌         | Interactive control-plane login in browser                                                 |
| [`vespa auth api-key`](https://docs.vespa.ai/en/reference/vespa-cli/vespa_auth_api-key.html) | ✅            | ❌         | Headless control-plane authentication with an API key generated in the Vespa Cloud console |
| [`vespa auth cert`](https://docs.vespa.ai/en/reference/vespa-cli/vespa_auth_cert.html)       | ❌            | ✅         | Used to generate a certificate for a data-plane connection                                 |
| [`VespaCloud`](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespacloud)       | ✅            | ❌         | `VespaCloud` is a control-plane connection to Vespa Cloud                                  |
| [`VespaDocker`](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespadocker)     | ✅            | ❌         | `VespaDocker` is a control-plane connection to a Vespa server running in Docker            |
| [`Vespa`](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa)                 | ❌            | ✅         | `Vespa` is a data-plane connection to an existing Vespa application                        |


## Defining your application

To initialize a connection to Vespa Cloud, you need to define your tenant name and application name.


In [3]:
# Replace with your tenant name from the Vespa Cloud Console
tenant_name = "vespa-team"
# Replace with your application name (does not need to exist yet)
application = "authdemo"

## Defining your application package

An [application package](https://docs.vespa.ai/en/application-packages.html) is the whole Vespa application configuration.
It can either be constructed directly from python (as we will do below) or initalized from a path, for example by cloning a sample application from the [Vespa sample apps](https://github.com/vespa-engine/sample-apps).

<div class="alert alert-info">
    Tip: You can use the command <a href="https://docs.vespa.ai/en/reference/vespa-cli/vespa_clone.html#examples">vespa clone album-recommendation my-app</a> to clone a single sample app if you have the Vespa CLI installed.
</div>

For this guide, we will create a minimal application package. See other guides for more complex examples.


In [5]:
from vespa.package import ApplicationPackage, Field, Schema, Document

package = ApplicationPackage(
    name=application,
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(
                        name="title",
                        type="string",
                        indexing=["index", "summary"],
                        index="enable-bm25",
                    ),
                    Field(
                        name="body",
                        type="string",
                        indexing=["index", "summary"],
                        index="enable-bm25",
                    ),
                ]
            ),
        )
    ],
)

## Control-plane authentication

Next, we need to authenticate to the Vespa Cloud control-plane.
There are two ways to authenticate to the control-plane:

### 1. **Interactive login**:

This is the recommended way to authenticate to the control-plane. It opens a browser window for you to authenticate with either google or github.

This method does not work on windows, currently. You can run `vespa auth login` in a terminal to authenticate first, and then use this method (which will then reuse the generated token).

(We will not run this method here, as the notebook is run in CI, but you should run it in your local environment)

```python
from vespa.deployment import VespaCloud

vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=application,
    application_package=package, # Could also initialize from application_root (path to application package)
)
```

You should see something similar to this:

```log
Checking for access token in auth.json...
Access token expired. Please re-authenticate.
Your Device Confirmation code is: DRDT-ZZDC
Automatically open confirmation page in your default browser? [Y/n] y
Opened link in your browser: https://vespa.auth0.com/activate?user_code=DRDT-ZZDC
Waiting for login to complete in browser ... done;1m⣯
Success: Logged in
 auth.json created at /Users/thomas/.vespa/auth.json
Successfully obtained access token for control plane access.
```


### 2. **API-key authentication**

This is a headless way to authenticate to the control-plane.

Note that the key must be generated, either with `vespa auth api-key` or in the Vespa Cloud console directly.


In [6]:
from vespa.deployment import VespaCloud
import os

# Key is only used for CI/CD. Can be removed if logging in interactively
key = os.getenv("VESPA_TEAM_API_KEY", None)
if key is not None:
    key = key.replace(r"\n", "\n")  # To parse key correctly


vespa_cloud = VespaCloud(
    tenant=tenant_name,  # Note that the name cannot contain the characters `-` or `_`.
    application=application,
    key_content=key,  # Prefer to use  key_location="<path-to-key-file.pem>"
    application_package=package,
)

Setting application...
Running: vespa config set application vespa-team.authdemo
Setting target cloud...
Running: vespa config set target cloud

Api-key found for control plane access. Using api-key.
Certificate and key not found in /Users/thomas/Repos/pyvespa/docs/sphinx/source/.vespa or /Users/thomas/.vespa/vespa-team.authdemo.default: Creating new cert/key pair with vespa CLI.
Generating certificate and key...
Running: vespa auth cert -N
Success: Certificate written to '/Users/thomas/.vespa/vespa-team.authdemo.default/data-plane-public-cert.pem'
Success: Private key written to '/Users/thomas/.vespa/vespa-team.authdemo.default/data-plane-private-key.pem'



When you have authenticated to the control-plane of Vespa Cloud, key/cert for data-plane authentication will be generated automatically for you, if none exists.
These will be a part of the application package that will be deployed, and you should keep them safe, as other users also will need them to authenticate to the data-plane of your Vespa application.

For `dev`-deployments, we allow redeploying an application with a different key/cert, than the previous deployment, but for `prod`-deployments, this is not allowed, and will require a `validation-overrides`-specification in the application package.


## Deploy to Vespa Cloud

The app is now defined and ready to deploy to Vespa Cloud.

Deploy `package` to Vespa Cloud, by creating an instance of
[VespaCloud](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.deployment.VespaCloud):


The following will upload the application package to Vespa Cloud Dev Zone (`aws-us-east-1c`), read more about [Vespa Zones](https://cloud.vespa.ai/en/reference/zones.html).
The Vespa Cloud Dev Zone is considered as a sandbox environment where resources are down-scaled and idle deployments are expired automatically.
For information about production deployments, see the following [example](https://pyvespa.readthedocs.io/en/latest/getting-started-pyvespa-cloud.html#Example:-Deploy-the-app-to-the-prod-environment).

> Note: Deployments to dev and perf expire after 7 days of inactivity, i.e., 7 days after running deploy. This applies to all plans, not only the Free Trial. Use the Vespa Console to extend the expiry period, or redeploy the application to add 7 more days.


In [8]:
app = vespa_cloud.deploy()

Deployment started in run 1 of dev-aws-us-east-1c for vespa-team.authdemo. This may take a few minutes the first time.
INFO    [09:14:37]  Deploying platform version 8.387.10 and application dev build 1 for dev-aws-us-east-1c of default ...
INFO    [09:14:37]  Using CA signed certificate version 1
INFO    [09:14:37]  Using 1 nodes in container cluster 'authdemo_container'
INFO    [09:14:39]  Using 1 nodes in container cluster 'authdemo_container'
INFO    [09:14:41]  Session 303896 for tenant 'vespa-team' prepared, but activation failed: 1/2 application hosts and 2/2 admin hosts for vespa-team.authdemo have completed provisioning and bootstrapping, still waiting for h95769.dev.aws-us-east-1c.vespa-external.aws.oath.cloud
INFO    [09:14:43]  Deploying platform version 8.387.10 and application dev build 1 for dev-aws-us-east-1c of default ...
INFO    [09:14:43]  1/2 application hosts and 2/2 admin hosts for vespa-team.authdemo have completed provisioning and bootstrapping, still waiting f

If the deployment failed, it is possible you forgot to add the key in the Vespa Cloud Console in the `vespa auth api-key` step above.

If you can authenticate, you should see lines like the following

```
 Deployment started in run 1 of dev-aws-us-east-1c for mytenant.hybridsearch.
```

The deployment takes a few minutes the first time while Vespa Cloud sets up the resources for your Vespa application

`app` now holds a reference to a [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) instance. We can access the
mTLS protected endpoint name using the control-plane (vespa_cloud) instance. This endpoint we can query and feed to (data plane access) using the
mTLS certificate generated in previous steps.


In [6]:
endpoint = vespa_cloud.get_mtls_endpoint()
endpoint

Found mtls endpoint for hybridsearch_container
URL: https://f7f73182.eb1181f2.z.vespa-app.cloud/


'https://f7f73182.eb1181f2.z.vespa-app.cloud/'

## Connect to a previously deployed application


### Feeding documents to Vespa

In this example we use the [HF Datasets](https://huggingface.co/docs/datasets/index) library to stream the
[BeIR/nfcorpus](https://huggingface.co/datasets/BeIR/nfcorpus) dataset and index in our newly deployed Vespa instance. Read
more about the [NFCorpus](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/):

> NFCorpus is a full-text English retrieval data set for Medical Information Retrieval.

The following uses the [stream](https://huggingface.co/docs/datasets/stream) option of datasets to stream the data without
downloading all the contents locally. The `map` functionality allows us to convert the
dataset fields into the expected feed format for `pyvespa` which expects a dict with the keys `id` and `fields`:

`{ "id": "vespa-document-id", "fields": {"vespa_field": "vespa-field-value"}}`


In [7]:
from datasets import load_dataset

dataset = load_dataset("BeIR/nfcorpus", "corpus", split="corpus", streaming=True)
vespa_feed = dataset.map(
    lambda x: {
        "id": x["_id"],
        "fields": {"title": x["title"], "body": x["text"], "id": x["_id"]},
    }
)

Now we can feed to Vespa using `feed_iterable` which accepts any `Iterable` and an optional callback function where we can
check the outcome of each operation. The application is configured to use [embedding](https://docs.vespa.ai/en/embedding.html)
functionality, that produce a vector embedding using a concatenation of the title and the body input fields. This step is resource intensive.

Read more about embedding inference in Vespa in the [Accelerating Transformer-based Embedding Retrieval with Vespa](https://blog.vespa.ai/accelerating-transformer-based-embedding-retrieval-with-vespa/)
blog post.

Default node resources in Vespa Cloud have 2 v-cpu for the Dev Zone.


In [8]:
from vespa.io import VespaResponse, VespaQueryResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(f"Error when feeding document {id}: {response.get_json()}")


app.feed_iterable(vespa_feed, schema="doc", namespace="tutorial", callback=callback)

Using mtls_key_cert Authentication against endpoint https://f7f73182.eb1181f2.z.vespa-app.cloud//ApplicationStatus


## Querying Vespa

Using the [Vespa Query language](https://docs.vespa.ai/en/query-language.html) we can query the indexed data.

- Using a context manager `with app.syncio() as session` to handle connection pooling ([best practices](https://cloud.vespa.ai/en/http-best-practices))
- The query method accepts any valid Vespa [query api parameter](https://docs.vespa.ai/en/reference/query-api-reference.html) in `**kwargs`
- Vespa api parameter names that contains `.` must be sent as `dict` parameters in the `body` method argument

The following searches for `How Fruits and Vegetables Can Treat Asthma?` using different retrieval and [ranking](https://docs.vespa.ai/en/ranking.html) strategies.

Query the text search app using the [Vespa Query language](https://docs.vespa.ai/en/query-language.html)
by sending the parameters to the body argument of
[Vespa.query](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa.query).

First we define a simple routine that will return a dataframe of the results for prettier display in the notebook.


In [9]:
import pandas as pd


def display_hits_as_df(response: VespaQueryResponse, fields) -> pd.DataFrame:
    records = []
    for hit in response.hits:
        record = {}
        for field in fields:
            record[field] = hit["fields"][field]
        records.append(record)
    return pd.DataFrame(records)

### Plain Keyword search

The following uses plain keyword search functionality with [bm25](https://docs.vespa.ai/en/reference/bm25.html) ranking, the `bm25` rank-profile was configured in the
application package to use a linear combination of the bm25 score of the query terms against the title and the body field.


In [10]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() limit 5",
        query=query,
        ranking="bm25",
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2450  Protective effect of fruits, vegetables and th...
1  MED-2464  Low vegetable intake is associated with allerg...
2  MED-1162  Pesticide residues in imported, organic, and "...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-2085  Antiplatelet, anticoagulant, and fibrinolytic ...


### Plain Semantic Search

The following uses dense vector representations of the query and the document and matching is performed and accelerated by Vespa's support for
[approximate nearest neighbor search](https://docs.vespa.ai/en/approximate-nn-hnsw.html).
The vector embedding representation of the text is obtained using Vespa's [embedder functionality](https://docs.vespa.ai/en/embedding.html#embedding-a-query-text).


In [11]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where ({targetHits:5}nearestNeighbor(embedding,q)) limit 5",
        query=query,
        ranking="semantic",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-5072  Lycopene-rich treatments modify noneosinophili...
1  MED-2472  Vegan regimen with reduced medication in the t...
2  MED-2464  Low vegetable intake is associated with allerg...
3  MED-2458  Manipulating antioxidant intake in asthma: a r...
4  MED-2450  Protective effect of fruits, vegetables and th...


### Hybrid Search

This is one approach to combine the two retrieval strategies and where we use Vespa's support for
[cross-hits feature normalization and reciprocal rank fusion](https://docs.vespa.ai/en/phased-ranking.html#cross-hit-normalization-including-reciprocal-rank-fusion). This
functionality is exposed in the context of `global` re-ranking, after the distributed query retrieval execution which might span 1000s of nodes.

#### Hybrid search with the OR query operator

This combines the two methods using logical disjunction (OR). Note that the first-phase expression in our `fusion` expression is only using the semantic score, this
because usually semantic search provides better recall than sparse keyword search alone.


In [12]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where userQuery() or ({targetHits:1000}nearestNeighbor(embedding,q)) limit 5",
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-2458  Manipulating antioxidant intake in asthma: a r...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-5072  Lycopene-rich treatments modify noneosinophili...


#### Hybrid search with the RANK query operator

This combines the two methods using the [rank](https://docs.vespa.ai/en/reference/query-language-reference.html#rank) query operator. In this case
we express that we want to retrieve the top-1000 documents using vector search, and then have sparse features like BM25 calculated as well (second operand
of the rank operator). Finally the hits are re-ranked using the reciprocal rank fusion


In [13]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql="select * from sources * where rank({targetHits:1000}nearestNeighbor(embedding,q), userQuery()) limit 5",
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-2458  Manipulating antioxidant intake in asthma: a r...
3  MED-2461  The association of diet with respiratory sympt...
4  MED-5072  Lycopene-rich treatments modify noneosinophili...


#### Hybrid search with filters

In this example we add another query term to the yql, restricting the nearest neighbor search to only consider documents that have vegetable in the title.


In [14]:
with app.syncio(connections=1) as session:
    query = "How Fruits and Vegetables Can Treat Asthma?"
    response: VespaQueryResponse = session.query(
        yql='select * from sources * where title contains "vegetable" and rank({targetHits:1000}nearestNeighbor(embedding,q), userQuery()) limit 5',
        query=query,
        ranking="fusion",
        body={"input.query(q)": f"embed({query})"},
    )
    assert response.is_successful()
    print(display_hits_as_df(response, ["id", "title"]))

         id                                              title
0  MED-2464  Low vegetable intake is associated with allerg...
1  MED-2450  Protective effect of fruits, vegetables and th...
2  MED-3199  Potential risks resulting from fruit/vegetable...
3  MED-2085  Antiplatelet, anticoagulant, and fibrinolytic ...
4  MED-4496  The effect of fruit and vegetable intake on ri...


## Next steps

This is just an intro into the capabilities of Vespa and pyvespa.
Browse the site to learn more about schemas, feeding and queries -
find more complex applications in
[examples](https://pyvespa.readthedocs.io/en/latest/examples.html).


## Example: Document operations using cert/key pair

Above, we deployed to Vespa Cloud, and as part of that, generated a data-plane mTLS cert/key pair.

This pair can be used to access the dataplane for reads/writes to documents and running queries from many different clients. The following
demonstrates that using the `requests` library.


Set up a dataplane connection using the cert/key pair:


In [15]:
import requests

cert_path = app.cert
key_path = app.key
session = requests.Session()
session.cert = (cert_path, key_path)

Get a document from the endpoint returned when we deployed to Vespa Cloud above. PyVespa wraps the Vespa [document api](https://docs.vespa.ai/en/document-v1-api-guide.html)
internally and in these examples we use the document api directly, but with the mTLS key/cert pair we used when deploying the app.


In [16]:
url = "{0}/document/v1/{1}/{2}/docid/{3}".format(endpoint, "tutorial", "doc", "MED-10")
doc = session.get(url).json()
doc

{'pathId': '/document/v1/tutorial/doc/docid/MED-10',
 'id': 'id:tutorial:doc::MED-10',
 'fields': {'body': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 par

Update the title and post the new version:


In [17]:
doc["fields"]["title"] = "Can you eat lobster?"
response = session.post(url, json=doc).json()
response

{'pathId': '/document/v1/tutorial/doc/docid/MED-10',
 'id': 'id:tutorial:doc::MED-10'}

Get the doc again to see the new title:


In [18]:
doc = session.get(url).json()
doc

{'pathId': '/document/v1/tutorial/doc/docid/MED-10',
 'id': 'id:tutorial:doc::MED-10',
 'fields': {'body': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 par

## Example: Reconnect pyvespa using cert/key pair

Above, we stored the dataplane credentials for later use. Deployment of an application usually happens when the schema changes, whereas accessing the dataplane is for document updates and user queries.

One only needs to know the endpoint and the cert/key pair to enable a connection to a Vespa Cloud application:


In [19]:
# cert_path = "/Users/me/.vespa/mytenant.hybridsearch.default/data-plane-public-cert.pem"
# key_path  = "/Users/me/.vespa/mytenant.hybridsearch.default/data-plane-private-key.pem"

from vespa.application import Vespa

the_app = Vespa(endpoint, cert=cert_path, key=key_path)

res = the_app.query(
    yql="select documentid, id, title from sources * where userQuery()",
    query="Can you eat lobster?",
    ranking="bm25",
)
res.hits[0]

Using mtls_key_cert Authentication against endpoint https://f7f73182.eb1181f2.z.vespa-app.cloud//ApplicationStatus


{'id': 'id:tutorial:doc::MED-10',
 'relevance': 25.27992205160453,
 'source': 'hybridsearch_content',
 'fields': {'documentid': 'id:tutorial:doc::MED-10',
  'id': 'MED-10',
  'title': 'Can you eat lobster?'}}

A common problem is a cert mismatch - the cert/key pair used when deployed is different than the pair used when making requests against Vespa. This
will cause 40x errors.

Make sure it is the same pair / re-create with `vespa auth cert -f` AND redeploy.

If you re-generate a mTLS certificate pair, and use that when connecting to Vespa cloud endpoint, it will fail until you have updaded the deployment with the new public certificate.

### Delete application

The following will delete the application and data from the dev environment.


In [20]:
vespa_cloud.delete()

Deactivated vespa-team.hybridsearch in dev.aws-us-east-1c
Deleted instance vespa-team.hybridsearch.default
